In [104]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.patches as patches
import numpy as np
import seaborn as sns
import os
import shutil
import re
from sklearn.model_selection import train_test_split

In [130]:
img_dir = './data/img/'
ann_dir = './data/ann/'

In [131]:
fileids = []
class_num = []
orient = []
img_fileids = []

for idx, img_fileid in enumerate(os.listdir(img_dir)):
    
    fileid = re.match('\w+_\d+', img_fileid).group(0)
    ann_fileid = fileid+'.json'
    
    img_fileids.append(img_fileid)
    fileids.append(fileid)
    with open(ann_dir+ann_fileid, "r", encoding="utf8") as f:
        contents = f.read() # string 타입
        json_data = json.loads(contents)
        class_num.append(json_data['objects'][0]['classTitle'])
        
        tags = json_data['objects'][0]['tags']
        for idx, tag in enumerate(tags):
            if tag['name'] == 'orientation':
                orient.append(tag['value'])
                
# for idx, ann_fileid in enumerate(os.listdir(ann_dir)):
#     fileid = re.match('\w+_\d+', ann_fileid).group(0)
#     fileids.append(fileid)
#     with open(ann_dir+ann_fileid, "r", encoding="utf8") as f:
#         contents = f.read() # string 타입
#         json_data = json.loads(contents)
#         class_num.append(json_data['objects'][0]['classTitle'])
        
#         tags = json_data['objects'][0]['tags']
#         for idx, tag in enumerate(tags):
#             if tag['name'] == 'orientation':
#                 orient.append(tag['value'])

In [132]:
df = pd.DataFrame()
df['fileid'] = fileids
df['img'] = img_fileids
df['class_num'] = class_num
df['orient'] = orient
df

,fileid,img,class_num,orient
0,self_0000,self_0000.JPG,1,Front
1,self_0001,self_0001.JPG,1,Rear
2,self_0002,self_0002.JPG,1,Front
3,self_0003,self_0003.JPG,1,Side
4,self_0004,self_0004.JPG,1,Side
...,...,...,...,...
1122,yt_2_0152,yt_2_0152.png,0,Side
1123,yt_2_0153,yt_2_0153.png,0,Side
1124,yt_2_0154,yt_2_0154.png,0,Front
1125,yt_2_0155,yt_2_0155.png,1,Side


In [133]:
df_0 = df.loc[df['class_num']=='0']
df_1 = df.loc[df['class_num']=='1']
df_2 = df.loc[df['class_num']=='2']
df_3 = df.loc[df['class_num']=='3']

In [141]:
data_0 = df_0[['img', 'fileid']]
target_0 = df_0['orient']
data_0, data_0_val, target_0, target_0_val = train_test_split(data_0, target_0, test_size=0.2, shuffle=True, stratify=target_0, random_state=1)
data_0_train, data_0_test, target_0_train, target_0_test = train_test_split(data_0, target_0, test_size=0.125, shuffle=True, stratify=target_0, random_state=1)

data_1 = df_1[['img', 'fileid']]
target_1 = df_1['orient']
data_1, data_1_val, target_1, target_1_val = train_test_split(data_1, target_1, test_size=0.2, shuffle=True, stratify=target_1, random_state=1)
data_1_train, data_1_test, target_1_train, target_1_test = train_test_split(data_1, target_1, test_size=0.125, shuffle=True, stratify=target_1, random_state=1)

data_2 = df_2[['img', 'fileid']]
target_2 = df_2['orient'] # 부족한 데이터 추가시 test_size 0.2->0.125 수정필요
data_2, data_2_val, target_2, target_2_val = train_test_split(data_2, target_2, test_size=0.2, shuffle=True, stratify=target_2, random_state=1)
data_2_train, data_2_test, target_2_train, target_2_test = train_test_split(data_2, target_2, test_size=0.2, shuffle=True, stratify=target_2, random_state=1)

data_3 = df_3[['img', 'fileid']]
target_3 = df_3['orient']
data_3, data_3_val, target_3, target_3_val = train_test_split(data_3, target_3, test_size=0.2, shuffle=True, stratify=target_3, random_state=1)
data_3_train, data_3_test, target_3_train, target_3_test = train_test_split(data_3, target_3, test_size=0.125, shuffle=True, stratify=target_3, random_state=1)

In [142]:
data_train = pd.concat([data_0_train, data_1_train, data_2_train, data_3_train])
data_val = pd.concat([data_0_val, data_1_val, data_2_val, data_3_val])
data_test = pd.concat([data_0_test, data_1_test, data_2_test, data_3_test])

In [143]:
data_train

,img,fileid
882,yt_1_0132.jpg,yt_1_0132
639,self_0640.jpg,self_0640
409,self_0410.jpg,self_0410
1016,yt_2_0042.png,yt_2_0042
878,yt_1_0128.jpg,yt_1_0128
...,...,...
102,self_0102.JPG,self_0102
185,self_0185.JPG,self_0185
558,self_0559.jpg,self_0559
968,yt_1_0218.jpg,yt_1_0218


In [155]:
for idx in data_train.index:
    img_path = data_train['img'][idx]
    ann_path = data_train['fileid'][idx]+'.json'
    
    img_origin = img_dir+img_path
    ann_origin = ann_dir+ann_path
    img_copy = './data/train/images/'+img_path
    ann_copy = './data/train/labels/'+ann_path
    shutil.copy(img_origin, img_copy)
    shutil.copy(ann_origin, ann_copy)

In [156]:
for idx in data_val.index:
    img_path = data_val['img'][idx]
    ann_path = data_val['fileid'][idx]+'.json'
    
    img_origin = img_dir+img_path
    ann_origin = ann_dir+ann_path
    img_copy = './data/val/images/'+img_path
    ann_copy = './data/val/labels/'+ann_path
    shutil.copy(img_origin, img_copy)
    shutil.copy(ann_origin, ann_copy)

In [157]:
for idx in data_test.index:
    img_path = data_test['img'][idx]
    ann_path = data_test['fileid'][idx]+'.json'
    
    img_origin = img_dir+img_path
    ann_origin = ann_dir+ann_path
    img_copy = './data/test/images/'+img_path
    ann_copy = './data/test/labels/'+ann_path
    shutil.copy(img_origin, img_copy)
    shutil.copy(ann_origin, ann_copy)

In [43]:
df.groupby(['class_num', 'orient']).count()

fileid
class_num orient        
0         Front       76
          Rear        70
          Side       274
1         Front      112
          Rear        74
          Side       384
2         Front        4
          Rear         2
          Side        11
3         Front       14
          Rear         4
          Side       102

In [35]:
df['orient']=='Rear'

0       False
1        True
2       False
3       False
4       False
        ...  
1122    False
1123    False
1124    False
1125    False
1126    False
Name: orient, Length: 1127, dtype: bool

In [38]:
df.loc[(df['orient']=='Rear') & (df['class_num']=='2')]

,fileid,class_num,orient
129,self_0129,2,Rear
134,self_0134,2,Rear


In [31]:
df.groupby(['class_num']).count()

,fileid,orient
class_num,,
0,420,420
1,570,570
2,17,17
3,120,120


In [32]:
data = df['Title']
target = df['class_num']
x, x_test, y, y_test = train_test_split(data, target, test_size=0.2, shuffle=True, stratify=target, random_state=1)

In [34]:
print(data.shape)
print(target.shape)
print(x.shape)
print(y.shape)

(1127,)
(1127,)
(901,)
(901,)


In [35]:
x_train, x_val, y_val, y_val = train_test_split(x, y, test_size=0.25, shuffle=True, stratify=y, random_state=1)

In [36]:
x_train

27      self_0027
1060    yt_2_0087
83      self_0083
227     self_0228
335     self_0336
          ...    
910     yt_1_0160
632     self_0633
788     yt_1_0038
460     self_0461
1095    yt_2_0124
Name: Title, Length: 675, dtype: object

In [37]:
x_val

1059    yt_2_0086
457     self_0458
52      self_0052
956     yt_1_0206
339     self_0340
          ...    
823     yt_1_0073
434     self_0435
356     self_0357
876     yt_1_0126
1114    yt_2_0144
Name: Title, Length: 226, dtype: object

In [38]:
x_test

279    self_0280
365    self_0366
563    self_0564
552    self_0553
444    self_0445
         ...    
7      self_0007
677    web_0034.
947    yt_1_0197
181    self_0181
923    yt_1_0173
Name: Title, Length: 226, dtype: object